In [4]:
# basic import
import os
import re
from os.path import join
from tqdm import tqdm
import math
import ast
import random
import warnings


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import cv2
import openslide
from PIL import Image
import torch

# spatialdata import 
import geopandas as gpd
import anndata
import spatialdata as sd
import squidpy as sq
import scanpy as sc
from shapely.geometry import Point
from spatialdata.models import Image2DModel, ShapesModel, TableModel
from spatialdata.transformations import (
    Affine,
    MapAxis,
    Scale,
    Sequence,
    Translation,
    get_transformation,
    set_transformation,
)

# MISO import 
from miso import Miso

In [5]:
# Setup device (GPU if available, otherwise CPU)
if torch.cuda.is_available():
    device = "cuda"
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU: NVIDIA GeForce RTX 4090 Laptop GPU


In [6]:
input_path = 'C:/Users/hua01/Desktop/cwc/SpatialMultimodal/spd_new/UNI'
output_path = 'C:/Users/hua01/Desktop/cwc/SpatialMultimodal/spd_new/miso'
subfolders = sorted([f.path for f in os.scandir(input_path) if f.is_dir()])
LEAP_folders  = [x for x in subfolders if '.zarr' in x]

In [7]:
sdata = sd.read_zarr(LEAP_folders[11])
adata = sdata['adata']
adata.X.toarray()

array([[1.5005606 , 0.        , 0.38155514, ..., 0.        , 0.        ,
        0.7707566 ],
       [1.6139451 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.3195369 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.43718192],
       ...,
       [1.3554627 , 0.2000201 , 0.        , ..., 0.        , 0.        ,
        0.36662355],
       [2.1712945 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.9917529 , 0.        , 0.3198194 , ..., 0.3198194 , 0.        ,
        0.17264116]], dtype=float32)

In [8]:
adata

AnnData object with n_obs × n_vars = 2118 × 6000
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'S_score', 'G2M_score', 'phase', '_indices', '_scvi_batch', '_scvi_labels', 'SampleID', 'LeapID', 'LEAP_PatientID', 'Treatment', 'ARM', 'pCR', 'gBRCA_status', 'PatientID', 'leiden_pca', 'leiden_scvi', 'Adipocytes_c2l_Midlevel', 'Arterial-like_EC_c2l_Midlevel', 'B_cell_c2l_Midlevel', 'CCR7_DC_c2l_Midlevel', 'CD4_Tfh-like_c2l_Midlevel', 'CD4_Treg_c2l_Midlevel', 'CD8_T_c2l_Midlevel', 'CD8_Texh_c2l_Midlevel', 'Cancer Epithelial_c2l_Midlevel', 'Capilary-like_EC_c2l_Midlevel', 'DC1_c2l_Midlevel', 'DC2_c2l_Midlevel', 'EC_angiogenic_c2l_Midlevel', 'Follicular_DC_c2l_Midlevel', 'GC_B_cell_c2l_Midlevel', 'LASP_c2l_Midlevel', 'LHS_c2l_Midlevel', 'Lymphatic_EC_c2l_Midlevel', 'Mac_C3_c2l_Midlevel', 'Mac_CCL18_c2l_Midlevel', 'Mac_CD36_c2l_Midlevel', 'Mac_COL27A1_c2l_Midlevel', 'Mac_LYVE1_c2l_Midlevel', 'Mac_SPP1_c2l_Midle

In [17]:
error_msg = []
for folder in LEAP_folders:
    try:
    # if True:
        leap_id = os.path.basename(folder)[:9]
        print(leap_id)
        sdata = sd.read_zarr(folder)
        adata = sdata["adata"]
        hne_emb = adata.obsm['hne_emb']
        model = Miso([adata.X.A, hne_emb],ind_views='all',combs='all',sparse=False,device=device)
        model.train()
        sdata['adata'].obsm['miso_emb'] = model.emb
        sdata.write(os.path.join(output_path, f"{leap_id}_sdata.zarr"), overwrite=True)

        color_key1 = "leiden_from_miso"
        color_key2 = "leiden_from_UNI"

        # 2. Create a figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 3. Generate each plot on its specific subplot axis
        sc.pp.neighbors(adata, use_rep="miso_emb", key_added='miso')
        sc.tl.leiden(adata, neighbors_key='miso', key_added=color_key1)
        sc.tl.umap(adata, neighbors_key='miso')
        # Plot 1: Spatial Scatter with 'leiden_from_miso'
        sq.pl.spatial_scatter(
            adata,
            color=color_key1,
            ax=axes[0, 0],      # Target the first subplot
            # show=False,    # REMOVE THIS LINE for sq.pl.spatial_scatter
            title=f"Spatial ({color_key1})"
        )
        
        # Plot 2: UMAP with 'leiden_from_miso'
        sc.pl.umap(
            adata,
            color=color_key1,
            ax=axes[0, 1],      # Target the second subplot
            show=False,      # Keep this for sc.pl.umap (it handles it)
            title=f"UMAP ({color_key1})"
        )

        sc.pp.neighbors(adata, use_rep="uni_emb", key_added='UNI')
        sc.tl.leiden(adata, neighbors_key='UNI', key_added=color_key2)
        sc.tl.umap(adata, neighbors_key='UNI')
        # Plot 3: Spatial Scatter with 'leiden_from_UNI'
        sq.pl.spatial_scatter(
            adata,
            color=color_key2,
            ax=axes[1, 0],      # Target the third subplot
            # show=False,    # REMOVE THIS LINE for sq.pl.spatial_scatter
            title=f"Spatial ({color_key2})"
        )
        
        # Plot 4: UMAP with 'leiden_from_UNI'
        sc.pp.neighbors(adata, use_rep="uni_emb", key_added='UNI')
        sc.tl.leiden(adata, neighbors_key='UNI', key_added=color_key2)
        sc.tl.umap(adata, neighbors_key='UNI')
        sc.pl.umap(
            adata,
            color=color_key2,
            ax=axes[1, 1],      # Target the second subplot
            show=False,      # Keep this for sc.pl.umap (it handles it)
            title=f"UMAP ({color_key2})"
        )
        
        # 4. Adjust layout
        plt.tight_layout()
    
        output_filename = os.path.join(output_path, f"{leap_id}_embeddings.jpg")
        # print(f"Saving combined plot to: {output_filename}")
        plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    
        plt.close(fig)
        
    except Exception as e:
        error_msg.append((leap_id, e))
        print((leap_id, e))
        continue

LEAP-002A


Training network for modality 1:   0%|          | 0/1000 [00:00<?, ?it/s]

Training network for modality 2:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\miso\LEAP-002A_sdata.zarr). Please 
         see the documentation of `is_self_contained()` to understand the implications of working with SpatialData 
         objects that are not self-contained.                                                                      
INFO     The Zarr backing store has been changed from                                                              
         C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\UNI\LEAP-002A_sdata.zarr the new file path:          
         C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\miso\LEAP-002A_sdata.zarr                            
LEAP-002B


Training network for modality 1:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Training network for modality 2:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\miso\LEAP-046B_sdata.zarr). Please 
         see the documentation of `is_self_contained()` to understand the implications of working with SpatialData 
         objects that are not self-contained.                                                                      
INFO     The Zarr backing store has been changed from                                                              
         C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\UNI\LEAP-046B_sdata.zarr the new file path:          
         C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\miso\LEAP-046B_sdata.zarr                            
LEAP-047B


Training network for modality 1:   0%|          | 0/1000 [00:00<?, ?it/s]

Training network for modality 2:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\miso\LEAP-047B_sdata.zarr). Please 
         see the documentation of `is_self_contained()` to understand the implications of working with SpatialData 
         objects that are not self-contained.                                                                      
INFO     The Zarr backing store has been changed from                                                              
         C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\UNI\LEAP-047B_sdata.zarr the new file path:          
         C:\Users\hua01\Desktop\cwc\SpatialMultimodal\spd_new\miso\LEAP-047B_sdata.zarr                            


In [18]:
error_msg

[]

In [19]:
len(LEAP_folders)

68